In [1]:
import os
import pandas as pd
import torch

import importlib
import pickle

base_path = os.getcwd()
data_path = os.path.join(base_path, "data")
question_path = os.path.join(data_path, "questions")
python_path = os.path.join(data_path, "python")

In [2]:
import ProblemSetReader
importlib.reload(ProblemSetReader)
problem_set_reader = ProblemSetReader.ProblemSetReader()
df = pd.read_pickle("./data/euler_df.pkl")

In [83]:
df

,idx,question,answer,python_code
0,1,If we list all the natural numbers below $10$ ...,233168,\n\ndef compute():\n\tans = sum(x for x in ran...
1,2,Each new term in the Fibonacci sequence is gen...,4613732,\n\ndef compute():\n\tans = 0\n\tx = 1 # Repr...
2,3,"The prime factors of $13195$ are $5, 7, 13$ an...",6857,\nimport math\n\n\ndef compute():\n\tn = 60085...
3,4,A palindromic number reads the same both ways....,906609,\n\ndef compute():\n\tans = max(i * j\n\t\tfor...
4,5,$2520$ is the smallest number that can be divi...,232792560,\nimport math\n\n\ndef compute():\n\tans = mat...
...,...,...,...,...
197,493,"$70$ coloured balls are placed in an urn, $10$...",6.818742,"\nimport fractions, math\n\n\ndef compute():\n..."
198,500,The number of divisors of $120$ is $16$.\nIn f...,35407281,"\nimport eulerlib, heapq\n\n\ndef compute():\n..."
199,518,Let $S(n) = \sum a + b + c$ over all triples $...,100315739184392,"\nimport eulerlib, math, itertools\n\n\ndef co..."
200,549,\nThe smallest number $m$ such that $10$ divid...,476001479068717,\nimport itertools\n\n\ndef compute():\n\tLIMI...


# Data Processing

In [ ]:
import DataProcessor
importlib.reload(DataProcessor)

In [ ]:
data_processor = DataProcessor.DataProcessor()

In [ ]:
df = data_processor.get_euler_dataframe()
# save df
df.to_pickle("/data/euler_df.pkl")

In [3]:
# load df from pickle
df = pd.read_pickle("./data/euler_df.pkl")

In [4]:
df

,idx,question,answer,python_code
0,1,If we list all the natural numbers below $10$ ...,233168,\n\ndef compute():\n\tans = sum(x for x in ran...
1,2,Each new term in the Fibonacci sequence is gen...,4613732,\n\ndef compute():\n\tans = 0\n\tx = 1 # Repr...
2,3,"The prime factors of $13195$ are $5, 7, 13$ an...",6857,\nimport math\n\n\ndef compute():\n\tn = 60085...
3,4,A palindromic number reads the same both ways....,906609,\n\ndef compute():\n\tans = max(i * j\n\t\tfor...
4,5,$2520$ is the smallest number that can be divi...,232792560,\nimport math\n\n\ndef compute():\n\tans = mat...
...,...,...,...,...
197,493,"$70$ coloured balls are placed in an urn, $10$...",6.818742,"\nimport fractions, math\n\n\ndef compute():\n..."
198,500,The number of divisors of $120$ is $16$.\nIn f...,35407281,"\nimport eulerlib, heapq\n\n\ndef compute():\n..."
199,518,Let $S(n) = \sum a + b + c$ over all triples $...,100315739184392,"\nimport eulerlib, math, itertools\n\n\ndef co..."
200,549,\nThe smallest number $m$ such that $10$ divid...,476001479068717,\nimport itertools\n\n\ndef compute():\n\tLIMI...


# Test CodeT5

In [ ]:
from transformers import T5ForConditionalGeneration, RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=10)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "user: {user.name}"

In [ ]:
tokenizer(text, return_tensors="pt")

In [ ]:
generated_ids

In [ ]:
tokenizer.decode(generated_ids[0][3])

In [ ]:
import ProblemSetReader
importlib.reload(ProblemSetReader)

In [ ]:
question, answer, code = problem_set_reader.get_problem_set(1)

In [ ]:
question

In [ ]:
input_ids = tokenizer(question, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=500)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# Test CodeGen

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

C:\Users\ChiehHung\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [73]:
size_str = "2B"
tokenizer = AutoTokenizer.from_pretrained(f"Salesforce/codegen-{size_str}-mono", padding_side="left")
model = AutoModelForCausalLM.from_pretrained(f"Salesforce/codegen-{size_str}-mono",
                                             output_hidden_states=False)

In [74]:
text = [problem_set_reader.get_question(i)+"\"\"\"" for i in range(1, 5)]
tokenizer.pad_token = tokenizer.eos_token

token_out = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
input_ids = token_out.input_ids
attention_mask = token_out.attention_mask

print(input_ids.shape)
print(attention_mask.shape)

torch.Size([4, 95])
torch.Size([4, 95])


In [75]:
print(input_ids[0])
print(attention_mask[0])

tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256,  1532,   356,  1351,   477,
          262,  3288,  3146,  2174,   720,   940,     3,   326,   389,  5021,
         2374,   286,   720,    18,     3,   393,   720,    20, 47113,   356,
          651,   720,    18,    11,   642,    11,   718,     3,   290,   720,
           24, 35307,   383,  2160,   286,   777,  5021,  2374,   318,   720,
         1954, 35307,   198, 16742,   262,  2160,   286,   477,   262,  5021,
         2374,   286,   720,    18,     3,   393,   720,    20,     3,  2174,
          720, 12825, 35307,   198, 37811])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 

In [76]:
completion = model.generate(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            max_length=512)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [77]:
completion.shape

torch.Size([4, 512])

In [78]:
decoded_texts = tokenizer.batch_decode(completion, skip_special_tokens=True)

In [80]:
for decoded_text in decoded_texts:
    print(decoded_text)

If we list all the natural numbers below $10$ that are multiples of $3$ or $5$, we get $3, 5, 6$ and $9$. The sum of these multiples is $23$.
Find the sum of all the multiples of $3$ or $5$ below $1000$.
"""

def sum_of_multiples(n):
    """
    >>> sum_of_multiples(10)
    23
    >>> sum_of_multiples(100)
    2318
    >>> sum_of_multiples(1000)
    231818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818

In [63]:
print(tokenizer.decode(completion[0]))

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>If we list all the natural numbers below $10$ that are multiples of $3$ or $5$, we get $3, 5, 6$ and $9$. The sum of these multiples is $23$.
Find the sum of all the multiples of $3$ or $5$ below $1000$.
"""

def sum_of_multiples(n):
    """
    >>> sum_of_multiples(10)
    23
    >>> sum_of_multiples(100)
    2318
    >>> sum_of_multiples(1000)
    2318181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818

In [81]:
print(tokenizer.decode(completion[0][len(input_ids[0]):][:-1]))



from math import sqrt

def is_prime(n):
    if n == 1:
        return False
    for i in range(2, int(sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def solution(limit):
    primes = []
    for i in range(2, limit):
        if is_prime(i):
            primes.append(i)
    
    max_prime = 0
    max_prime_sum = 0
    for i in range(len(primes)):
        for j in range(i, len(primes)):
            sum = primes[i] + primes[j]
            if sum < limit:
                if is_prime(sum):
                    if sum > max_prime_sum:
                        max_prime_sum = sum
                        max_prime = primes[i] + primes[j]
    return max_prime

print(solution(1000000))



In [65]:
tokenizer_out = tokenizer(problem_set_reader.get_question(1)+"\"\"\"", return_tensors="pt", 
                          padding=True, truncation=True)
model.generate(input_ids=tokenizer_out.input_ids,
               attention_mask=tokenizer_out.attention_mask,
               max_length=512)
print(tokenizer.decode(completion[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>If we list all the natural numbers below $10$ that are multiples of $3$ or $5$, we get $3, 5, 6$ and $9$. The sum of these multiples is $23$.
Find the sum of all the multiples of $3$ or $5$ below $1000$.
"""

def sum_of_multiples(n):
    """
    >>> sum_of_multiples(10)
    23
    >>> sum_of_multiples(100)
    2318
    >>> sum_of_multiples(1000)
    2318181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818

In [68]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")

completion = model.generate(**tokenizer(problem_set_reader.get_question(1)+"\"\"\"", return_tensors="pt"), max_length=256)

print(tokenizer.decode(completion[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


If we list all the natural numbers below $10$ that are multiples of $3$ or $5$, we get $3, 5, 6$ and $9$. The sum of these multiples is $23$.
Find the sum of all the multiples of $3$ or $5$ below $1000$.
"""

def sum_of_multiples(n):
    """
    >>> sum_of_multiples(10)
    23
    >>> sum_of_multiples(100)
    2318
    >>> sum_of_multiples(1000)
    23181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818


In [84]:
generated_path = os.path.join(os.getcwd(), "data", "generated")
if not os.path.isdir(generated_path):
    os.mkdir(generated_path)

In [85]:
idx = 50
generated_code_str = tokenizer.decode(completion[0][len(input_ids[0]):][:-1])

with open(os.path.join(generated_path, f"p{idx:03d}.py"), "w", encoding="utf-8") as f:
    f.write(generated_code_str)
    

# Evaluator

In [3]:
import sys
from io import StringIO

code = open(os.path.join(python_path, "p001.py"), "r").read()

old_stdout = sys.stdout
redirected_output = sys.stdout = StringIO()
try: 
    exec(code)
except:
    raise Exception("Error in code")
finally: 
    sys.stdout = old_stdout

answer_calculated = redirected_output.getvalue()
answer_calculated

'233168\n'

In [9]:
import Evaluator
importlib.reload(Evaluator)
evaluator = Evaluator.Evaluator()

In [10]:
evaluator.evaluate(os.path.join(python_path, "p001.py"), problem_set_reader.get_answer(1))

True